In [1]:
import pandas as pd 
df_train = pd.read_hdf('../data/processed/train/with_zeros.hdf')
df_train.head()

,ID,shop_id,item_id,year,month
0,0,5,5037,2014,10
1,1,5,5320,2014,10
2,2,5,5233,2014,10
3,3,5,5232,2014,10
4,4,5,5268,2014,10


In [7]:
df_train['date'] = pd.to_datetime(df_train['year'].astype(str) + '-' + df_train['month'].astype(str) + '-1')

In [28]:
df_train['7_days_bef'] = df_train['date'] - pd.DateOffset(days=7)
df_train.head()

,ID,shop_id,item_id,year,month,date,7_days_bef
0,0,5,5037,2014,10,2014-10-01,2014-09-24
1,1,5,5320,2014,10,2014-10-01,2014-09-24
2,2,5,5233,2014,10,2014-10-01,2014-09-24
3,3,5,5232,2014,10,2014-10-01,2014-09-24
4,4,5,5268,2014,10,2014-10-01,2014-09-24


In [28]:
df_aggr_train = pd.read_hdf('../data/processed/train/aggr_train_zeros.hdf')
df_aggr_train.head()

,ID,shop_id,item_id,year,month,item_name,item_category_id,item_category_name,first_big_category,last_big_category,...,count_aggr_2_quartals_ago_city_first_big_category,count_aggr_year_city_first_big_category,count_aggr_1_month_ago_city_last_big_category,count_aggr_1_month_city_last_big_category,count_aggr_3_months_city_last_big_category,count_aggr_6_months_city_last_big_category,count_aggr_2_quartals_ago_city_last_big_category,count_aggr_year_city_last_big_category,fridays,item_cnt_day
0,0,5,5037,2014,10,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,PS3,...,1780.0,6923.0,139.0,89.0,332.0,587.0,382.0,1482.0,5,0.0
1,2,5,5233,2014,10,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,Игры,PS3,...,1780.0,6923.0,139.0,89.0,332.0,587.0,382.0,1482.0,5,0.0
2,15,5,4872,2014,10,"Minecraft. Playstation 3 Edition [PS3, русская...",19,Игры - PS3,Игры,PS3,...,1780.0,6923.0,139.0,89.0,332.0,587.0,382.0,1482.0,5,3.0
3,21,5,5002,2014,10,"NBA 2K16 [PS3, русская документация]",19,Игры - PS3,Игры,PS3,...,1780.0,6923.0,139.0,89.0,332.0,587.0,382.0,1482.0,5,0.0
4,23,5,5814,2014,10,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,Игры - PS3,Игры,PS3,...,1780.0,6923.0,139.0,89.0,332.0,587.0,382.0,1482.0,5,2.0


In [3]:
def get_weekly_sales(df_ids, df_train_raw, keys):
    df_train_raw['next_month'] = df_train_raw['date'] + pd.DateOffset(months=1)
    df_train_raw['next_month_date'] = pd.to_datetime(
    df_train_raw['next_month'].dt.year.astype(str) + '-' + df_train_raw['next_month'].dt.month.astype(str) + '-1'
)
    df_train_raw['month_end_date'] = df_train_raw['next_month_date'] - pd.DateOffset(days=7)
    df_train_raw = df_train_raw[df_train_raw['date'] >= df_train_raw['month_end_date']]
    weekly_sales = df_train_raw.groupby(keys + ['next_month_date'], as_index=False)['item_cnt_day'].sum().rename(
        columns={'item_cnt_day' : 'count_last_week_' + '_'.join(keys)}
    )
    df_train_weekly = pd.merge(df_ids, weekly_sales,
             left_on=keys + ['date'],
             right_on=keys + ['next_month_date'],
             how='left'
            ).drop('next_month_date', axis=1).fillna(0)
    return df_train_weekly
    

In [6]:
df = get_weekly_sales(df, df_raw, ['shop_id', 'item_id'])

In [7]:
df['days_in_a_month'] = df['date'].dt.daysinmonth
df['expected_sales'] = df['count_last_week_shop_id_item_id'] / 7 * df['days_in_a_month']

In [8]:
from sklearn.metrics import r2_score
r2_score(df['item_cnt_day'], df['expected_sales'])

KeyError: 'item_cnt_day'

In [38]:
r2_score(df['item_cnt_day'], df['count_aggr_1_month_shop_id_item_id'])

0.3369327128701297

In [10]:
df.to_hdf('../data/processed/test/aggr_test_manual_feat.hdf', 'test')

In [2]:
import pandas as pd 
df = pd.read_hdf('../data/processed/test/aggr_test.hdf')

In [9]:
df['count_last_week_shop_id_item_id'].hist()
import matplotlib.pyplot as plt 
plt.show()

<Figure size 640x480 with 1 Axes>

In [23]:
from calendar import monthrange
monthrange(2018, 5)[1]

31

In [27]:
df['date'].dt.daysinmonth.value_counts()

31    1713600
30     856800
28     214200
Name: date, dtype: int64

In [22]:
df['count_last_week_shop_id_item_id'].value_counts()

 0.0      2668105
 1.0        89268
 2.0        15328
 3.0         4940
 4.0         2312
 5.0         1258
 6.0          715
 7.0          480
 8.0          331
 9.0          240
 10.0         192
 11.0         158
 12.0         133
 13.0         132
-1.0          121
 14.0          88
 15.0          76
 16.0          62
 17.0          56
 19.0          50
 18.0          49
 21.0          43
 22.0          33
 20.0          32
 23.0          30
 25.0          24
 24.0          21
 32.0          19
 26.0          18
 27.0          18
           ...   
 170.0          1
 177.0          1
 179.0          1
 202.0          1
 238.0          1
 269.0          1
 321.0          1
 137.0          1
 97.0           1
 104.0          1
 71.0           1
 62.0           1
 487.0          1
 65.0           1
 66.0           1
 68.0           1
 48.0           1
 69.0           1
 46.0           1
 77.0           1
 100.0          1
 78.0           1
 79.0           1
 80.0           1
 82.0     

In [5]:
df_raw = pd.read_csv('../data/raw/sales_train.csv', parse_dates=['date'])
df_raw.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [32]:
df_raw['next_month'] = df_raw['date'] + pd.DateOffset(months=1)
df_raw['next_month_date'] = pd.to_datetime(
    df_raw['next_month'].dt.year.astype(str) + '-' + df_raw['next_month'].dt.month.astype(str) + '-1'
)
df_raw['month_end_date'] = df_raw['next_month_date'] - pd.DateOffset(days=7)

In [33]:
df_raw[df_raw['date'] >= df_raw['month_end_date']].shape

(713377, 29)

In [35]:
weekly_sales = df_raw.groupby(['shop_id', 'item_id', 'next_month_date'], as_index=False)['item_cnt_day'].sum()

In [37]:
df_train.head()

,ID,shop_id,item_id,year,month,date,7_days_bef
0,0,5,5037,2014,10,2014-10-01,2014-09-24
1,1,5,5320,2014,10,2014-10-01,2014-09-24
2,2,5,5233,2014,10,2014-10-01,2014-09-24
3,3,5,5232,2014,10,2014-10-01,2014-09-24
4,4,5,5268,2014,10,2014-10-01,2014-09-24


In [38]:
df_train_weekly = pd.merge(df_train, weekly_sales,
         left_on=['shop_id', 'item_id', 'date'],
         right_on=['shop_id', 'item_id', 'next_month_date'],
         how='left'
        ).fillna(0)
df_train_weekly.head()

,ID,shop_id,item_id,year,month,date,7_days_bef,next_month_date,item_cnt_day
0,0,5,5037,2014,10,2014-10-01,2014-09-24,2014-10-01 00:00:00,1.0
1,1,5,5320,2014,10,2014-10-01,2014-09-24,0,0.0
2,2,5,5233,2014,10,2014-10-01,2014-09-24,0,0.0
3,3,5,5232,2014,10,2014-10-01,2014-09-24,0,0.0
4,4,5,5268,2014,10,2014-10-01,2014-09-24,0,0.0


In [17]:
df_raw['date'].head()

0    2013-01-02
1    2013-01-03
2    2013-01-05
3    2013-01-06
4    2013-01-15
Name: date, dtype: object

In [21]:
df_raw['7_days_bef'] = df_raw['date

0           2
1           3
2           5
3           6
4          15
5          10
6           2
7           4
8          11
9           3
10          3
11          5
12          7
13          8
14         10
15         11
16         13
17         16
18         26
19         27
20          9
21         16
22         27
23         27
24         29
25         27
26          6
27         26
28          2
29          6
           ..
2935819    16
2935820    22
2935821    22
2935822    17
2935823    26
2935824    14
2935825    23
2935826     4
2935827    18
2935828    21
2935829     3
2935830    11
2935831    22
2935832    26
2935833    22
2935834    29
2935835    18
2935836    22
2935837    24
2935838    17
2935839    24
2935840    31
2935841    11
2935842    10
2935843     9
2935844    10
2935845     9
2935846    14
2935847    22
2935848     3
Name: date, Length: 2935849, dtype: int64

In [4]:
for i1, i2 in df_train.iterrows():
    a = 2

KeyboardInterrupt: 

In [5]:
df_raw[['shop_id', 'item_id']].drop_duplicates().shape

(424124, 2)

In [7]:
pd.to_datetime(df_raw['date']).dt.year

KeyboardInterrupt: 